In [20]:
# From:
#   https://hdfgroup.org/wp/2015/03/from-hdf5-datasets-to-apache-spark-rdds/
# Specialized to MIDS W205-2 (Fall 2015)

# Modules needed below:
#   Pandas
#   h5py
#   psycopg2
#   MySQL-python

import os
import os.path
import h5py
import sys

mainRoot = os.path.expanduser('~/MIDS/mss/') 
mssRoot = os.path.expanduser('~/MIDS/mss/data/')
blocksRoot = os.path.expanduser('~/MIDS/mss/blocks/')
resultsRoot = os.path.expanduser('~/MIDS/mss/results/')

llss = os.listdir(mainRoot)

if blocksRoot.split('/')[-2] not in llss:
    os.mkdir(blocksRoot)
if resultsRoot.split('/')[-2] not in llss:
    os.mkdir(resultsRoot)
if mssRoot.split('/')[-2] not in llss:
    os.mkdir(mssRoot)
    
    

filesPerBlock = 100  # Gives about 30MB/file. Should use larger size in production.

minPartitions = 4 #minimum number of partitions for use by Spark


In [21]:
# Configure spark to work
import os
import sys

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

#execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

In [22]:
def getFileNames(dataRoot, verbose = True):
	
	fileStructure = os.walk(dataRoot)
	
	h5Files = []
	for entry in fileStructure:
		thisPath = entry[0]
		thisFileList = entry[2]
		#print thisPath, thisFileList

		for ff in thisFileList:
			if ff.endswith('.h5'):
				h5Files.append(os.path.join(thisPath,ff))
				if verbose:
					print os.path.join(thisPath,ff)
	return h5Files

In [23]:
def buildReferenceCSV(dataRoot = None, publish = True, verbose = True):
	'''
	"Walks" the file tree and finds all of the HD5 files in the tree under dataRoot
	(assumes these are from the million song database)
	'''
	
	if not dataRoot:
		dataRoot = blocksRoot
    
	h5Files = getFileNames(dataRoot, verbose = verbose)

	if publish:
		csvFileName = os.path.join(dataRoot,'Hd5Extracts.csv')
		oFile = open(csvFileName, 'w')
        
	
	for ff in h5Files:

		pubString = ff
#		pubString += ', '
#		pubString += 'analysis, '
#		pubString += 'key \n'   # Key of the song, not key of the hash

		if publish:
			oFile.write(pubString + '\n')

		if verbose:
			print pubString

	return csvFileName

In [37]:
def nodeToDict(inputLine, verbose = False):
    '''
    This function is intended for use in Spark.
    Inputs a line (probably from the reference CSV)
    and extracts some important info 
    
    Returns a list of dictionaries, each dictionary has all "analysis" fields in the song.
    It's possible that all songs don't have exactly the same fields.
    '''
    import h5py
    parsedLine = inputLine.split(',')
    thisH5File = h5py.File(parsedLine[0])
    
    if parsedLine[0].split('/')[-1].startswith('Chunk'):
        thisBlockSongList = thisH5File['songs']

        songInfo = []

        for thisSong in thisBlockSongList:
            # There are 3 categories of data in this set.  Grab stuff from all
            # of them
            analysisDataSet = thisH5File['songs'][thisSong]['analysis']['songs']
            metadataDataSet = thisH5File['songs'][thisSong]['metadata']['songs']
            musicbrainzDataSet = thisH5File['songs'][thisSong]['musicbrainz']['songs']

            outDict = dict()

            for measurement1 in analysisDataSet.dtype.names:
                value = analysisDataSet[measurement1][0]
                
                if len(str(value)) > 0:
                    outDict[measurement1] = value
                else:
                    outDict[measurement1] = 'NA'

                

            for measurement2 in musicbrainzDataSet.dtype.names:
                value = musicbrainzDataSet[measurement2][0]
    
                if len(str(value)) > 0:
                    outDict[measurement2] = value
                else:
                    outDict[measurement2] = 'NA'


            for measurement3 in metadataDataSet.dtype.names:
                value = metadataDataSet[measurement3][0]            ##

                if len(str(value)) > 0:
                    outDict[measurement3] = value
                else:
                    outDict[measurement3] = 'NA'
                
            #if verbose:
            #print outTuple
            songInfo.append(outDict)

        return songInfo


In [39]:
#  Process in Spark

from pyspark import SparkContext
from optparse import OptionParser
from datetime import datetime as dt
print 'Establishing Spark Context'

# Kill the context if it already exists
if 'sc' in dir():
    sc.stop()

sc = SparkContext(appName="SparkHDF5")

csvFileName = buildReferenceCSV(dataRoot = blocksRoot, verbose = False)

print 'Fetching all file names'
timeNow = dt.now().isoformat()
file_paths = sc.textFile(os.path.join(blocksRoot,'Hd5Extracts.csv'),minPartitions=minPartitions)

print 'Mapping RDD'
rdd = file_paths.flatMap(nodeToDict)
results = rdd.take(1000)

print 'Results Generated.'

sc.stop()




Establishing Spark Context
Fetching all file names
Mapping RDD
Results Generated.


In [40]:
# Get unique column names
# and write results out as a csv

import csv

allParameters = []

# This loop gets all the names of the measurements (key, tempo,etc) making this code 
# not dependent on every song having exactly the same structure
for song in results:
    [allParameters.append(measure) for measure in song.keys()]
    allParameters = list(set(allParameters))   # keep only unique measurement names 
    
# Write dictionaries as .csv file
with open(os.path.join(resultsRoot,'MSD_Flat.csv'), 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, allParameters)
    dict_writer.writeheader()
    dict_writer.writerows(results)

print '\n\n\tComplete.  CSV written to ', os.path.join(resultsRoot,'MSD_Flat.csv'), '\n\n'



	Complete.  CSV written to  /home/james/MIDS/mss/results/MSD_Flat.csv 




In [43]:
# Import the flat csv file into pandas
# This will facilitate any hands-on analyses we might want to do
# And makes setting up a postgres table REALLY easy

import MySQLdb
import pandas as pd

# Connect to a local MySQL server using the username "james" (no password) 
# and the database mss.
dbConnection=MySQLdb.connect(host="localhost",user="james",db="mss")

# Turn our csv file into a Pandas data frame.  
df = pd.read_csv(os.path.join(resultsRoot,'MSD_Flat.csv'))

# Tell pandas to load this dataframe as a MySQL table
df2 = df.astype(object).where(pd.notnull(df), None)
df2.to_sql("MSS", con= dbConnection, flavor = 'mysql', if_exists = 'replace')



/usr/lib/python2.7/dist-packages/pandas/io/sql.py:264: Warning: Data truncated for column 'release' at row 9
  cur.executemany(insert_query, data)
/usr/lib/python2.7/dist-packages/pandas/io/sql.py:264: Warning: Data truncated for column 'artist_location' at row 44
  cur.executemany(insert_query, data)
/usr/lib/python2.7/dist-packages/pandas/io/sql.py:264: Warning: Data truncated for column 'title' at row 94
  cur.executemany(insert_query, data)
/usr/lib/python2.7/dist-packages/pandas/io/sql.py:264: Warning: Data truncated for column 'title' at row 167
  cur.executemany(insert_query, data)
/usr/lib/python2.7/dist-packages/pandas/io/sql.py:264: Warning: Data truncated for column 'artist_name' at row 217
  cur.executemany(insert_query, data)
/usr/lib/python2.7/dist-packages/pandas/io/sql.py:264: Warning: Data truncated for column 'artist_name' at row 233
  cur.executemany(insert_query, data)
/usr/lib/python2.7/dist-packages/pandas/io/sql.py:264: Warning: Data truncated for column 'title' 

In [ ]:
import pylab as plt
plt.plot(df.key,df.tempo,'.')
plt.show()

In [44]:
import numpy as np
#rownames = ['C','C#','D','Eb','E','F','F#','G','Ab','A','Bb','B']
pd.crosstab(df.key,df.time_signature)

time_signature,1,3,4,5,7
key,,,,,
0,24,8,71,5,1
1,9,15,55,7,1
2,20,15,77,7,4
3,6,5,16,3,0
4,16,16,52,4,2
5,8,14,42,5,3
6,9,9,38,3,3
7,20,13,86,8,4
8,11,7,34,1,0


In [100]:
df

,key,mode_confidence,idx_tatums_start,key_confidence,energy,idx_bars_confidence,idx_beats_confidence,idx_segments_pitches,time_signature,year,duration,artist_mbid,artist_location,artist_7digitalid,idx_artist_terms,time_signature_confidence,artist_longitude,idx_segments_loudness_max_time,idx_sections_start,artist_playmeid,
0,1,0.858,0,0.888,0,0,0,0,4,0,201.09016,5700dcd4-c139-4f31-aa3e-6382b9af9032,"D�orf, Germany",983,0,0.968,NaN,0,0,881,...
1,8,0.465,0,0.319,0,0,0,0,4,2006,222.74567,4a8ac23a-e27a-44db-814a-f8c31beb68e6,"New London, CT",27822,0,0.774,-72.09647,0,0,4664,...
2,5,0.524,0,0.643,0,0,0,0,4,0,329.63873,NaN,NaN,399673,0,1.000,NaN,0,0,-1,...
3,4,0.234,0,0.000,0,0,0,0,1,1974,195.65669,2d67f7f4-1e85-4278-bac6-424e6204a8b9,NaN,108554,0,0.619,NaN,0,0,38963,...
4,0,0.713,0,0.904,0,0,0,0,4,2008,147.93098,1f223eed-aa38-49da-9e76-62ab3adc2e04,"New York, NY [The Bronx]",10549,0,0.350,-73.85678,0,0,7529,...
5,2,0.146,0,0.000,0,0,0,0,3,0,486.94812,e54203ac-f4ee-4970-a54f-78993f8b91a1,England,2352,0,0.922,-1.97406,0,0,18452,...
6,9,0.626,0,0.530,0,0,0,0,4,0,242.62485,7e92079b-6d93-4c8d-87bd-364c3a5d2bc3,"Yorkshire, England",367053,0,0.000,-1.20788,0,0,-1,...
7,9,0.416,0,0.634,0,0,0,0,1,0,191.24200,52889ea9-1c52-487e-95a4-8c53f8322ca2,"Ramsey, Fayette County, IL",22402,0,0.986,-89.10827,0,0,98091,...
8,0,0.239,0,0.000,0,0,0,0,4,1960,143.35955,c8d0d74b-80eb-4746-8cea-f5df78b4b90c,"McLellan, FL",2090,0,0.000,NaN,0,0,16411,...
9,9,0.228,0,0.000,0,0,0,0,4,1979,216.65914,74dca2c7-d7dc-4b13-bd17-502a35db042a,NaN,21344,0,0.841,NaN,0,0,34193,...


In [3]:
# Tell pandas to load this dataframe as a MySQL table
import pandas as pd
import os
import os.path
dfBillboard = pd.read_csv(os.path.join(resultsRoot,'billboard.csv'))
dfBillboard = df.astype(object).where(pd.notnull(df), None)
dfBillboard.to_sql("billboard", con= dbConnection, flavor = 'mysql', if_exists = 'replace')


NameError: name 'resultsRoot' is not defined